works with https://tech.yandex.ru/maps/geosearch/doc/concepts/request-docpage/

In [59]:
import pandas as pd
import folium
import requests
import os

### importing data

In [60]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# link for mapbox map as a underlay for folium
map_url = api_keys.loc['mapbox_map']['key']

# API Yandex organization search
search_api_key = api_keys.loc['yandex_search']['key']

In [61]:
# coords table
coords = pd.read_excel('./input_params.xlsx', sheet_name = 'coords')
coords.set_index('point_name', inplace=True)

def extract_lon_lat(location):
    loc = coords.loc[location]
    return loc['lon'], loc['lat']

# center point coords
center_lon, center_lat = extract_lon_lat('center')

# south-west bounding box coordinates
sw_lon, sw_lat = extract_lon_lat('southwest')

# north-east bounding box coordinates
ne_lon, ne_lat = extract_lon_lat('northeast')

In [62]:
# table of items to request
request_list = pd.read_excel('./input_params.xlsx', sheet_name = 'request_example')


In [63]:
# lists of items without missing values
for item_type in list(request_list.columns):
    list_of_items = request_list[item_type].dropna().to_list()
    print(list_of_items)

['школа', 'детский сад']
['поликлиника', 'больница']
['стадион', 'бассейн', 'каток']
['кинотеатр', 'концертный зал', 'музей', 'театр']
['супермаркет', 'магазин одежды', 'магазин обуви', 'рынок', 'книжный магазин']
['парикмахерская', 'ремонт обуви', 'ремонт одежды', 'банк', 'прачечная']
['ресторан', 'кафе', 'бар']


### request

In [64]:
def organization_request(item_type):
    # list of queries from column
    list_of_items = request_list[item_type].dropna().to_list()
    
    # request parameters - URL and bounding box

    URL = 'https://search-maps.yandex.ru/v1/'
    bbox = f"{sw_lon:.6f},{sw_lat:.6f}~{ne_lon:.6f},{ne_lat:.6f}"
        
    results = []
    
    # request
    for search_query in list_of_items:
        for skip_cnt in range(0, 25000, 500):
            params = { 
                'text' : search_query, 
                'apikey': search_api_key,
                'lang': 'ru_RU',
                'type':'biz',
                'bbox': bbox,
                'rspn': 1,
                'results': 500,
                'skip': skip_cnt
            }
            response = requests.get(URL, params=params)
            response_json = response.json()
            results += response_json['features']
            feature_count = len(response_json['features'])
            if feature_count < 500:
                break

    # take only unique values is results

    result_dict = {}
    for result in results:
        result_dict[result['properties']['CompanyMetaData']['id']] = result
    results_unique = list(result_dict.values())

    # transform json, delete exceed columns, create category columns
    for res in results_unique:
        res.update(res['properties'])
        res['lon'], res['lat'] = res['geometry']['coordinates']
        res.update(res['CompanyMetaData'])
        res['category'] = res['Categories'][0]['class']
        res['category_name'] = res['Categories'][0]['name']
        for cat in res['Categories']:
            res['cat_' + cat['class'].replace(' ', '_')] = True
        for key in ['properties', 'CompanyMetaData', 'boundedBy', 'type', 'geometry', 'url', 'Phones', 'Hours', 'Categories']:
            if key in res:
                del res[key]

    # create dataframe out of transformed results
    df = pd.DataFrame(results_unique)
    
    return df

### saving result

In [65]:
# create dir for output files
os.makedirs("output", exist_ok=True)

In [66]:
# create csv files for every category in request

for item_type in list(request_list.columns):
    df = organization_request(item_type)
    df.to_csv("./output/{}.csv".format(str(item_type))) 

In [67]:
# example for single category

school_df = organization_request('education')
school_df.to_csv('./output/school_example.csv')
school_df.head()

,address,cat_college,cat_fallback_services,cat_fitness,cat_kindergarten,cat_medicine,cat_office_service,category,category_name,description,id,lat,lon,name
0,"Россия, Иркутская область, Ангарск, 33-й микро...",True,True,NaN,NaN,NaN,NaN,college,Школа танцев,"11В, 33-й микрорайон, Ангарск, Россия",1742078974,52.514210,103.845447,Dancer
1,"Россия, Иркутская область, Ангарск, 215-й квар...",True,NaN,NaN,NaN,NaN,NaN,college,Школа танцев,"50, 215-й квартал, Ангарск, Россия",77915148804,52.498861,103.838827,Танцевальный центр Universal Dance
2,"Россия, Иркутская обл., Ангарск г., микрорайон...",True,NaN,True,NaN,NaN,NaN,college,Школа танцев,"19А, 18-й микрорайон, Ангарск, Россия",1024261901,52.502231,103.845143,New Star
3,"Россия, Иркутская область, Ангарск, улица Круп...",True,NaN,NaN,True,NaN,NaN,kindergarten,Детский сад,"ул. Крупской, 3, Ангарск, Россия",1065757131,52.522851,103.879805,Начальная школа - детский сад № 1
4,"Россия, Иркутская область, Ангарск, 10-й микро...",True,NaN,NaN,NaN,NaN,True,college,Курсы иностранных языков,"46, 10-й микрорайон, Ангарск, Россия",1239065369,52.508620,103.861849,Языковая школа Лингва Сити
